In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np

In [ ]:
import cv2
for i in os.listdir("../input/flowers-recognition/flowers/daisy/")[:5]:
    img = cv2.imread("../input/flowers-recognition/flowers/daisy/"+i)
    print(img.shape)

In [ ]:
path = "../input/flowers-recognition/flowers/flowers/"
inception = MobileNet(input_shape= [150,150,3], weights='imagenet', include_top=False)

In [ ]:
for layer in inception.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(inception.output)
prediction = Dense(5, activation='softmax')(x)
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                  validation_split=0.2)

In [ ]:
train = datagen.flow_from_directory(path,target_size=(150,150),batch_size=32,class_mode='categorical',subset='training')
test = datagen.flow_from_directory(path,target_size=(150,150),batch_size=32,class_mode='categorical',subset='validation')

In [ ]:
model.compile(  loss='categorical_crossentropy',  optimizer='adam',  metrics=['accuracy'])

In [ ]:
r = model.fit_generator(
  train,
  validation_data=test,
  epochs=10,
  steps_per_epoch=len(train),
  validation_steps=len(test)
)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
model.save('flower_classification.h5')